## Use subprocess to manage child processes
Child processes started by Python are able to run in parallel, enabling you to use Python to consume all of the CPU cores of your machine.

In [5]:
import subprocess

proc = subprocess.Popen(["echo", "Hello from the child!"], stdout=subprocess.PIPE, shell=True)
out, err = proc.communicate()
print(out.decode("utf-8"))

"Hello from the child!"

